# Simple Techniques
1. Max Voting
2. Averaging
3. Weighted Averaging

## 1. Max Voting
Max voting is generally used for classification problems. In this technique, multiple models are used to make predictions for each data point. The predictions by each model are considered a **vote**. The predictions which we get from the majority of the models are used as the final prediction.

For example, say you asked 5 friends to rate a movie (out of 5) and the results were:

| Friend 1  | Friend 2  | Friend 3  | Friend 4  | Friend 5  | Final Rating  |
|---|---|---|---|---|---|
| 5  | 4  | 5  | 4  | 4  | 4  |

### Sample Code

In [18]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np

data = load_iris()
X = data.data
y = data.target
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.3)

lgr = LogisticRegression(solver='lbfgs', multi_class='auto')
knc = KNeighborsClassifier()
dtc = DecisionTreeClassifier()

lgr.fit(X_train, y_train)
knc.fit(X_train, y_train)
dtc.fit(X_train, y_train)

lgr_pred = lgr.predict(X_test)
knc_pred = knc.predict(X_test)
dtc_pred = dtc.predict(X_test)

final_pred = np.array([])

for i in range(0, len(X_test)):
    final_pred = np.append(final_pred, ([lgr_pred[i], knc_pred[i], dtc_pred[i]]))
    
# final_pred = our final predictions

Alternatively, we can use scitkit's `VotingClassifier`:

In [19]:
from sklearn.ensemble import VotingClassifier

model1 = LogisticRegression(solver='lbfgs', multi_class='auto', random_state=1)
model2 = DecisionTreeClassifier(random_state=1)

model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')

model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9047619047619048

# 2. Averaging
Similar to max voting, multiple predictions are made for each point in averaging. In this method, we take an average of predictions from all the models and use it to make the final predictions. Averaging can be used for making predictions in regression problems or while calculating the probabilities for classification problems.

For our "friend" example:
| Friend 1  | Friend 2  | Friend 3  | Friend 4  | Friend 5  | Final Rating  |
|---|---|---|---|---|---|
| 5  | 4  | 5  | 4  | 4  | 4.4  |

$Final Prediction = \frac{5+4+5+4+4}{5} = 4.4$

### Sample Code
This is the same code used as above, instead now averaging the results:

In [23]:
final_average_pred = (lgr_pred + knc_pred + dtc_pred) / 3

# final_average_pred = our final predictions
# this isn't great here since we're looking for categorical responses

# 3. Weighted Average
This is an extension of the averaging method. All models are assigned different weights defining the importance of each model for prediction. For instance, if two of your friends are critics while others have no prior experience, then the answers by the critics are given more importance:

| _| Friend 1  | Friend 2  | Friend 3  | Friend 4  | Friend 5  | Final Rating  |
|---|---|---|---|---|---|---|
| weight | 0.23 | 0.23 | 0.18 | 0.18 | 0.18 |  |
| rating | 5  | 4  | 5  | 4  | 4  |  4.41 |

$$Final Rating = \Big[(5 \times 0.23)+(4 \times 0.23)+(5 \times 0.18)+(4 \times 0.18)+(4 \times 0.18)\Big] = 4.41$$

## Sample Code
Again, same data from the max voting example:

In [25]:
final_pred_wa = (lgr_pred*0.4 + dtc_pred*0.3, knc_pred*0.3)

# final_pred_wa